In [1]:
PRETRAINED_MODEL_ID = "google/bigbird-roberta-base"
SPLIT = ["train[:1%]", "validation[:1%]"] # change it later

from transformers import BigBirdForQuestionAnswering, BigBirdTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

In [2]:
tokenizer = BigBirdTokenizer.from_pretrained(PRETRAINED_MODEL_ID)
data = load_dataset("natural_questions", split=SPLIT)
tr_data, val_data = data
print(data)

Using custom data configuration default
Reusing dataset natural_questions (/home/vasu/.cache/huggingface/datasets/natural_questions/default/0.0.2/867dbbaf9137c1b83ecb19f5eb80559e1002ea26e702c6b919cfa81a17a8c531)
[Dataset({
    features: ['id', 'document', 'question', 'annotations'],
    num_rows: 3074
}), Dataset({
    features: ['id', 'document', 'question', 'annotations'],
    num_rows: 78
})]


In [20]:
def get_single_answer(example, is_long_answer=False):
    def choose_first(answer):
        assert isinstance(answer, list)
        if len(answer) == 1:
            return answer[0]
        for a in answer:
            if is_long_answer:
                a['start_token'] = [a['start_token']]
            if len(a['start_token']) > 0:
                break
        return a

    answer = {}
    if example['annotations']['yes_no_answer'] in [0, 1]:
        answer['category'] = '[YES]' if example['annotations']['yes_no_answer'] == 1 else '[NO]'
        answer['start_token'] = answer['end_token'] = 0
        answer['start_byte'] = answer['end_byte'] = 0
        answer['text'] = '[CLS]'
    else:
        answer['category'] = "[SHORT]"
        o = choose_first(example['annotations']['short_answers'])
        print(example['id'], o)
        assert isinstance(o, dict)
        if len(o['start_token']) == 0:
            answer['category'] = "[LONG]"
            o = choose_first(choose_first(example['annotations']['long_answer']))
        answer.update({k:o[k] for k in o})

    return {"answer": answer}

In [21]:
    example = val_data[0]
    example['answer'] = get_single_answer(example)

# def preprocess_nq(example):
    question = example['question']['text']
    context = example['document']['tokens']
    context = " ".join(context['token'][i] for i in range(len(context["token"])) if not context['is_html'][i])
    inputs = tokenizer(question, context)
    # return inputs

5225754983651766092 {'end_byte': [44657], 'end_token': [159], 'start_byte': [44318], 'start_token': [140], 'text': ['enabled European empire expansion into the Americas and trade routes to become established across the Atlantic and Pacific oceans']}


In [22]:
o = val_data.map(get_single_answer)

5225754983651766092 {'end_byte': [44657], 'end_token': [159], 'start_byte': [44318], 'start_token': [140], 'text': ['enabled European empire expansion into the Americas and trade routes to become established across the Atlantic and Pacific oceans']}


5225754983651766092 {'end_byte': [44657], 'end_token': [159], 'start_byte': [44318], 'start_token': [140], 'text': ['enabled European empire expansion into the Americas and trade routes to become established across the Atlantic and Pacific oceans']}
6986236841860957647 {'end_byte': [], 'end_token': [], 'start_byte': [], 'start_token': [], 'text': []}



TypeError: object of type 'int' has no len()

In [15]:
o

Dataset({
    features: ['annotations', 'answer', 'document', 'id', 'question'],
    num_rows: 78
})

In [16]:
o["answer"]

[{'category': '[SHORT]',
  'end_byte': [44657],
  'end_token': [159],
  'start_byte': [44318],
  'start_token': [140],
  'text': ['enabled European empire expansion into the Americas and trade routes to become established across the Atlantic and Pacific oceans']},
 {'category': '[SHORT]',
  'end_byte': [],
  'end_token': [],
  'start_byte': [],
  'start_token': [],
  'text': []},
 {'category': '[SHORT]',
  'end_byte': [37946],
  'end_token': [251],
  'start_byte': [37907],
  'start_token': [245],
  'text': ['Wilhelm Conrad Röntgen, of Germany']},
 {'category': '[SHORT]',
  'end_byte': [],
  'end_token': [],
  'start_byte': [],
  'start_token': [],
  'text': []},
 {'category': '[SHORT]',
  'end_byte': [62818],
  'end_token': [535],
  'start_byte': [62806],
  'start_token': [531],
  'text': ['May 18, 2018']},
 {'category': '[SHORT]',
  'end_byte': [],
  'end_token': [],
  'start_byte': [],
  'start_token': [],
  'text': []},
 {'category': '[SHORT]',
  'end_byte': [],
  'end_token': [],
 

In [158]:
example['answer']

{'category': '[SHORT]',
 'end_byte': [44657],
 'end_token': [159],
 'start_byte': [44318],
 'start_token': [140],
 'text': ['enabled European empire expansion into the Americas and trade routes to become established across the Atlantic and Pacific oceans']}

In [86]:
# a = preprocess_nq(tr_data[0])
question, context[:1000]

('mexican leader who was supported by the united states during mexican civil war',
 "United States involvement in the Mexican Revolution - wikipedia United States involvement in the Mexican Revolution Jump to : navigation , search See also : Mexico - United States relations United States involvement in the Mexican Revolution Part of the Mexican Revolution Aftermath of Pancho Villa 's attack on Columbus , New Mexico , in 1916 Date 1910 - 1919 Location Mexico , United States Belligerents United States Huertistas Villistas Constitutionalistas Carrancistas Maderistas Commanders and leaders Woodrow Wilson John J. Pershing Frank Friday Fletcher Victoriano Huerta Pancho Villa Alvaro Obregon Venustiano Carranza Francisco Madero United States involvement in the Mexican Revolution Mexican Revolution Tampico Affair Ypiranga incident Veracruz Border War 1st Agua Prieta 1st Ciudad Juarez Bandit War Norias Ranch Ojo de Agua 2nd Nogales Santa Isabel Columbus Mexican Expedition Guerrero Agua Caliente 

In [87]:
(tokenizer.decode(inputs.input_ids))[:1000]

"[CLS] mexican leader who was supported by the united states during mexican civil war[SEP] United States involvement in the Mexican Revolution - wikipedia United States involvement in the Mexican Revolution Jump to : navigation, search See also : Mexico - United States relations United States involvement in the Mexican Revolution Part of the Mexican Revolution Aftermath of Pancho Villa's attack on Columbus, New Mexico, in 1916 Date 1910 - 1919 Location Mexico, United States Belligerents United States Huertistas Villistas Constitutionalistas Carrancistas Maderistas Commanders and leaders Woodrow Wilson John J. Pershing Frank Friday Fletcher Victoriano Huerta Pancho Villa Alvaro Obregon Venustiano Carranza Francisco Madero United States involvement in the Mexican Revolution Mexican Revolution Tampico Affair Ypiranga incident Veracruz Border War 1st Agua Prieta 1st Ciudad Juarez Bandit War Norias Ranch Ojo de Agua 2nd Nogales Santa Isabel Columbus Mexican Expedition Guerrero Agua Caliente

In [88]:
example.keys()

dict_keys(['id', 'document', 'question', 'annotations'])

In [89]:
example['annotations'].keys()

dict_keys(['id', 'long_answer', 'short_answers', 'yes_no_answer'])

In [90]:
example['annotations']['long_answer']

[{'end_byte': 73287,
  'end_token': 790,
  'start_byte': 69526,
  'start_token': 374}]

In [91]:
example['annotations']['short_answers']

[{'end_byte': [70383],
  'end_token': [453],
  'start_byte': [70369],
  'start_token': [451],
  'text': ['Benito Juárez']}]

In [69]:
example['annotations']['yes_no_answer']

[-1]

In [123]:
tr_data[0]['annotations']

{'id': ['14208778017604803798'],
 'long_answer': [{'end_byte': 73287,
   'end_token': 790,
   'start_byte': 69526,
   'start_token': 374}],
 'short_answers': [{'end_byte': [70383],
   'end_token': [453],
   'start_byte': [70369],
   'start_token': [451],
   'text': ['Benito Juárez']}],
 'yes_no_answer': [-1]}

In [122]:
val_data[0]['annotations']

{'id': ['4323936797498927989',
  '13037645000009169623',
  '4439059471919323171',
  '15051359051424338858',
  '5332861748513580580'],
 'long_answer': [{'end_byte': -1,
   'end_token': -1,
   'start_byte': -1,
   'start_token': -1},
  {'end_byte': -1, 'end_token': -1, 'start_byte': -1, 'start_token': -1},
  {'end_byte': -1, 'end_token': -1, 'start_byte': -1, 'start_token': -1},
  {'end_byte': 44666,
   'end_token': 161,
   'start_byte': 43178,
   'start_token': 44},
  {'end_byte': 44666,
   'end_token': 161,
   'start_byte': 43178,
   'start_token': 44}],
 'short_answers': [{'end_byte': [],
   'end_token': [],
   'start_byte': [],
   'start_token': [],
   'text': []},
  {'end_byte': [],
   'end_token': [],
   'start_byte': [],
   'start_token': [],
   'text': []},
  {'end_byte': [],
   'end_token': [],
   'start_byte': [],
   'start_token': [],
   'text': []},
  {'end_byte': [44657],
   'end_token': [159],
   'start_byte': [44318],
   'start_token': [140],
   'text': ['enabled European 

In [105]:
[val_data[i]['annotations'] for i in range(10)]

[{'id': ['4323936797498927989',
   '13037645000009169623',
   '4439059471919323171',
   '15051359051424338858',
   '5332861748513580580'],
  'long_answer': [{'end_byte': -1,
    'end_token': -1,
    'start_byte': -1,
    'start_token': -1},
   {'end_byte': -1, 'end_token': -1, 'start_byte': -1, 'start_token': -1},
   {'end_byte': -1, 'end_token': -1, 'start_byte': -1, 'start_token': -1},
   {'end_byte': 44666,
    'end_token': 161,
    'start_byte': 43178,
    'start_token': 44},
   {'end_byte': 44666,
    'end_token': 161,
    'start_byte': 43178,
    'start_token': 44}],
  'short_answers': [{'end_byte': [],
    'end_token': [],
    'start_byte': [],
    'start_token': [],
    'text': []},
   {'end_byte': [],
    'end_token': [],
    'start_byte': [],
    'start_token': [],
    'text': []},
   {'end_byte': [],
    'end_token': [],
    'start_byte': [],
    'start_token': [],
    'text': []},
   {'end_byte': [44657],
    'end_token': [159],
    'start_byte': [44318],
    'start_token':

In [149]:
[tr_data[i]['annotations']['yes_no_answer'] for i in range(1000)]

[[-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
 [-1],
